In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import itertools
from tqdm import tqdm

In [2]:
transform = transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

In [3]:
size_hidden_layer = [5, 10,50, 100, 500, 1000]
size_train_iter = [10,20, 50, 100, 250, 500, 750, 1000, 1500, 2000]
full_accur_list = []
full_time_list = []

In [4]:
trainset = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
valset = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
for N in tqdm(size_hidden_layer):
    
    accur_list = []
    time_list = []
    
    for n_training in tqdm(size_train_iter):
        input_size = 784
        hidden_sizes = [N, N]
        output_size = 10

        model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                              nn.ReLU(),
                              nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                              nn.ReLU(),
                              nn.Linear(hidden_sizes[1], output_size),
                              nn.LogSoftmax(dim=1))
        model.to(device)
        print(model)
        criterion = nn.NLLLoss() #nn.MSELoss()

        optimizer = optim.Adam(model.parameters(),lr=0.001)
        time0 = time()
        epochs = 15
        for e in range(epochs):
            running_loss = 0
            for images, labels in itertools.islice(trainloader, n_training):
                # Flatten MNIST images into a 784 long vector
                images, labels = images.to(device), labels.to(device)
                images = images.view(images.shape[0], -1)

                # Training pass
                optimizer.zero_grad()

                output = model(images)
                loss = criterion(output, labels)

                #This is where the model learns by backpropagating
                loss.backward()

                #And optimizes its weights here
                optimizer.step()

                running_loss += loss.item()
            else:
                print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))

        print("\nTraining Time (in minutes) =",(time()-time0)/60)
        time_list += [(time()-time0)/60]
        correct_count, all_count = 0, 0
        for images,labels in valloader:
            images, labels = images.to(device), labels.to(device)
            for i in range(len(labels)):
                img = images[i].view(1, 784)
                with torch.no_grad():
                    logps = model(img)


                ps = torch.exp(logps)
                probab = list(ps.cpu().numpy()[0])
                pred_label = probab.index(max(probab))
                true_label = labels.cpu().numpy()[i]
                if(true_label == pred_label):
                    correct_count += 1
                all_count += 1

        print("Number Of Images Tested =", all_count)
        print("\nModel Accuracy =", (correct_count/all_count))
        accur_list += [correct_count/all_count]

    full_accur_list += [accur_list]
    full_time_list += [time_list]
    np.savetxt('full_accur_list2.txt', np.array(full_accur_list))
    np.savetxt('full_time_list2.txt', np.array(full_time_list))
    np.savetxt('size_hidden_layer2.txt', np.array(size_hidden_layer))
    np.savetxt('size_train_iter2.txt', np.array(size_train_iter))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Sequential(
  (0): Linear(in_features=784, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=10, bias=True)
  (5): LogSoftmax()
)
Epoch 0 - Training loss: 0.024874646526409874
Epoch 1 - Training loss: 0.02437087086472176
Epoch 2 - Training loss: 0.023766443927659156
Epoch 3 - Training loss: 0.023349015189132204
Epoch 4 - Training loss: 0.02315657352333638
Epoch 5 - Training loss: 0.022955198531974353
Epoch 6 - Training loss: 0.022888254255119925
Epoch 7 - Training loss: 0.022073501589964194
Epoch 8 - Training loss: 0.022099804268208647
Epoch 9 - Training loss: 0.021430731073879737
Epoch 10 - Training loss: 0.021103272941320944
Epoch 11 - Training loss: 0.02084445038329818
Epoch 12 - Training loss: 0.02047479216223841
Epoch 13 - Training loss: 0.019961310093845133
Epoch 14 - Training loss: 0.019160592988101657

Training Time (in minutes) = 0.04621682961781819



 10%|████████▎                                                                          | 1/10 [00:12<01:51, 12.35s/it]

Number Of Images Tested = 10000

Model Accuracy = 0.3928
Sequential(
  (0): Linear(in_features=784, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=10, bias=True)
  (5): LogSoftmax()
)
Epoch 0 - Training loss: 0.04897663359448854
Epoch 1 - Training loss: 0.04682761290942682
Epoch 2 - Training loss: 0.04556864589007933
Epoch 3 - Training loss: 0.04412311175738825
Epoch 4 - Training loss: 0.04280904577230848
Epoch 5 - Training loss: 0.04157266344851268
Epoch 6 - Training loss: 0.04072317144255648
Epoch 7 - Training loss: 0.04037837280647587
Epoch 8 - Training loss: 0.03832116792959445
Epoch 9 - Training loss: 0.038043019868163414
Epoch 10 - Training loss: 0.03659692438426557
Epoch 11 - Training loss: 0.035903632386661034
Epoch 12 - Training loss: 0.034926479559209044
Epoch 13 - Training loss: 0.034050910838885604
Epoch 14 - Training loss: 0.03287986804173191

Training Time (in minutes


 20%|████████████████▌                                                                  | 2/10 [00:23<01:36, 12.09s/it]

...

Number Of Images Tested = 10000

Model Accuracy = 0.9284
Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=10, bias=True)
  (5): LogSoftmax()
)
Epoch 0 - Training loss: 0.10592616023793658
Epoch 1 - Training loss: 0.04321289454092349
Epoch 2 - Training loss: 0.037638356515974886
Epoch 3 - Training loss: 0.0346296976433634
Epoch 4 - Training loss: 0.032430236491916786
Epoch 5 - Training loss: 0.030419296753813208
Epoch 6 - Training loss: 0.027980441859027726
Epoch 7 - Training loss: 0.02520696744958221
Epoch 8 - Training loss: 0.023757730084441618
Epoch 9 - Training loss: 0.02312718381855025
Epoch 10 - Training loss: 0.02135310491233238
Epoch 11 - Training loss: 0.02003385611911064
Epoch 12 - Training loss: 0.018529556091946325
Epoch 13 - Training loss: 0.018786683070983714
Epoch 14 - Training loss: 0.018303873307351618

Training Time 


 40%|█████████████████████████████████▏                                                 | 4/10 [01:18<01:59, 19.85s/it]

Number Of Images Tested = 10000

Model Accuracy = 0.9457
Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=10, bias=True)
  (5): LogSoftmax()
)
Epoch 0 - Training loss: 0.17497072562670657
Epoch 1 - Training loss: 0.08768155652958193
Epoch 2 - Training loss: 0.07317118647732715
Epoch 3 - Training loss: 0.058819254467100984
Epoch 4 - Training loss: 0.053057097097131994
Epoch 5 - Training loss: 0.04739387449361622
Epoch 6 - Training loss: 0.042012846887683565
Epoch 7 - Training loss: 0.03972513044773261
Epoch 8 - Training loss: 0.037109739828243185
Epoch 9 - Training loss: 0.03235533917302897
Epoch 10 - Training loss: 0.032068977755969015
Epoch 11 - Training loss: 0.02965386807997979
Epoch 12 - Training loss: 0.028940858758255236
Epoch 13 - Training loss: 0.026820587915684115
Epoch 14 - Training loss: 0.02423504680824051

Training Time 


 50%|█████████████████████████████████████████▌                                         | 5/10 [02:15<02:34, 30.94s/it]

Number Of Images Tested = 10000

Model Accuracy = 0.9662
Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=10, bias=True)
  (5): LogSoftmax()
)
Epoch 0 - Training loss: 0.2562634689149572
Epoch 1 - Training loss: 0.1419059824285858
Epoch 2 - Training loss: 0.10548528476453412
Epoch 3 - Training loss: 0.08575238304129287
Epoch 4 - Training loss: 0.07393963750142024
Epoch 5 - Training loss: 0.06389071639857567
Epoch 6 - Training loss: 0.05821805378235479
Epoch 7 - Training loss: 0.056356529385518675
Epoch 8 - Training loss: 0.05089830509674892
